# MIMIC-III query API tutorial

This notebook shows examples of how to use the cyclops.query API on [MIMIC-III v1.4](https://physionet.org/content/mimiciii/1.4/).

* First, setup the MIMIC-III database according to the instructions in [mimic-code](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iii/buildmimic/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `MIMICIIIQuerier` accordingly.

## Imports and instantiate `MIMICIIIQuerier`

In [ ]:
import cyclops.query.ops as qo
from cyclops.query import MIMICIIIQuerier

querier = MIMICIIIQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="mimiciii",
    user="postgres",
    password="pwd",
)
# List all custom table methods.
querier.list_custom_tables()

## Example 1. Get all male patients with a mortality outcome.

In [ ]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("expire_flag", 1),
        qo.ConditionEquals("gender", "M"),
    ]
)
patients = querier.mimiciii.patients(ops=ops).run()
print(f"{len(patients)} rows extracted!")

## Example 2. Get all female patient encounters with diagnoses (`gastroenteritis` in ICD-9 long title).

In [ ]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("gender", "F"),
    ]
)
diagnoses_ops = qo.Sequential(
    [
        qo.ConditionSubstring("long_title", "gastroenteritis"),
    ]
)
patients = querier.mimiciii.patients(ops=ops)
admissions = querier.mimiciii.admissions(
    join=qo.JoinArgs(join_table=patients.query, on="subject_id"),
)
diagnoses = querier.diagnoses(
    join=qo.JoinArgs(
        join_table=admissions.query,
        on=["subject_id", "hadm_id"],
    ),
    ops=diagnoses_ops,
).run()
print(f"{len(diagnoses)} rows extracted!")

## Example 3. Get potassium lab tests for female patient encounters.

In [ ]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("gender", "F"),
    ]
)
patients = querier.mimiciii.patients(ops=ops)
lab_ops = qo.Sequential(
    [
        qo.ConditionEquals("label", "potassium"),
    ]
)
labs = querier.labevents(ops=lab_ops).run()
print(f"{len(labs)} rows extracted!")

## Example 4. Get AaDO2 carevue chart events for male patients that have a `valuenum` of less than 20.

In [ ]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("gender", "M"),
    ]
)
chartevents_ops = qo.Sequential(
    [
        qo.ConditionEquals("dbsource", "carevue"),
        qo.ConditionEquals("label", "AaDO2"),
        qo.ConditionLessThan("valuenum", 20),
    ]
)
patients = querier.mimiciii.patients(ops=ops)
chart_events = querier.chartevents(ops=chartevents_ops).run()
print(f"{len(chart_events)} rows extracted!")